<a href="https://colab.research.google.com/github/WeruHarrison/ml-zoomcamp-homeworks/blob/main/08_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from torchsummary import summary

## Download and Unzip Data

In [2]:
url = "https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip"
output_file = "data.zip"
!wget {url} -O {output_file}

# unzip the data
!unzip -q {output_file}


--2025-12-02 07:14:40--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-02T07%3A52%3A51Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-02T06%3A52%3A11Z&ske=2025-12-02T07%3A52%3A51Z&sks=b&skv=2018-11-09&sig=YFOhre8Z0VXAIC80QS9haNKo9QsLlvtugmADXftuTsw%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDY2MTQ4MCwibmJmIjoxNzY0NjU5NjgwLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG

## Set Reproducibility Seeds

In [3]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Data preparation



In [4]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = datasets.ImageFolder("/content/data/train", transform=train_transforms)
validation_dataset = datasets.ImageFolder("/content/data/test", transform=test_transforms)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

print("Train samples:", len(train_dataset))
print("Test samples:", len(validation_dataset))
print("Classes:", train_dataset.classes)


Train samples: 800
Test samples: 201
Classes: ['curly', 'straight']


# Question 1: Which loss function you will use?
 Answer: I will use BCEWithLogitsLoss() because this is binary classification problem.


## Model definition

In [5]:
class HairTypeCNN(nn.Module):
    def __init__(self):
        super(HairTypeCNN, self).__init__()

        # Convolution layer
        self.conv = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=0
        )

        # ReLU
        self.relu = nn.ReLU()

        # MaxPool 2x2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten_dim = 32 * 99 * 99

        # Linear layers
        self.fc1 = nn.Linear(self.flatten_dim, 64)
        self.fc2 = nn.Linear(64, 1)  # Binary classification output

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, self.flatten_dim)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)  # raw logits
        return x

# Create model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HairTypeCNN().to(device)

# Loss & optimizer
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

model


HairTypeCNN(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

## Training

In [6]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")


Epoch 1/10, Loss: 0.6462, Acc: 0.6362, Val Loss: 0.6032, Val Acc: 0.6517
Epoch 2/10, Loss: 0.5475, Acc: 0.7100, Val Loss: 0.7251, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5533, Acc: 0.7250, Val Loss: 0.5991, Val Acc: 0.6716
Epoch 4/10, Loss: 0.4802, Acc: 0.7712, Val Loss: 0.6033, Val Acc: 0.6567
Epoch 5/10, Loss: 0.4334, Acc: 0.8025, Val Loss: 0.6196, Val Acc: 0.6766
Epoch 6/10, Loss: 0.3740, Acc: 0.8325, Val Loss: 0.7371, Val Acc: 0.6766
Epoch 7/10, Loss: 0.2721, Acc: 0.8838, Val Loss: 0.9223, Val Acc: 0.6418
Epoch 8/10, Loss: 0.2478, Acc: 0.9000, Val Loss: 0.7294, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2075, Acc: 0.9200, Val Loss: 0.7523, Val Acc: 0.7015
Epoch 10/10, Loss: 0.1494, Acc: 0.9450, Val Loss: 0.7894, Val Acc: 0.7015


# Question 2: What's the total number of parameters of the model?

In [7]:
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

Total parameters: 20073473


In [8]:
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------


# Question 3: What is the median of training accuracy for all the epochs for this model?

In [9]:
median_acc = np.median(history['acc'])
median_acc

np.float64(0.8175)

# Question 4: What is the standard deviation of training loss for all the epochs for this model?

In [10]:
std_loss = np.std(history['loss'])
std_loss

np.float64(0.15896418235516768)

## Data augmentation

In [11]:
augmented_train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Reload only the training dataset with augmentation
augmented_train_dataset = datasets.ImageFolder(
    "/content/data/train",
    transform=augmented_train_transforms
)

augmented_train_loader = DataLoader(
    augmented_train_dataset,
    batch_size=20,
    shuffle=True
)

print("Augmented train samples:", len(augmented_train_dataset))


Augmented train samples: 800


## Continued training the model with augmentation

In [12]:
num_epochs_aug = 10  # another 10 epochs

for epoch in range(num_epochs_aug):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in augmented_train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(augmented_train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"[AUG] Epoch {epoch+1}/{num_epochs_aug}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")


[AUG] Epoch 1/10, Loss: 0.6984, Acc: 0.6288, Val Loss: 0.5601, Val Acc: 0.7264
[AUG] Epoch 2/10, Loss: 0.5891, Acc: 0.7013, Val Loss: 0.6733, Val Acc: 0.6816
[AUG] Epoch 3/10, Loss: 0.5380, Acc: 0.7188, Val Loss: 0.5335, Val Acc: 0.7512
[AUG] Epoch 4/10, Loss: 0.5146, Acc: 0.7462, Val Loss: 0.5935, Val Acc: 0.7065
[AUG] Epoch 5/10, Loss: 0.5237, Acc: 0.7350, Val Loss: 0.5339, Val Acc: 0.7463
[AUG] Epoch 6/10, Loss: 0.4892, Acc: 0.7662, Val Loss: 0.6110, Val Acc: 0.6915
[AUG] Epoch 7/10, Loss: 0.4865, Acc: 0.7700, Val Loss: 0.5350, Val Acc: 0.7413
[AUG] Epoch 8/10, Loss: 0.5010, Acc: 0.7500, Val Loss: 0.6925, Val Acc: 0.6219
[AUG] Epoch 9/10, Loss: 0.4707, Acc: 0.7738, Val Loss: 0.6901, Val Acc: 0.6716
[AUG] Epoch 10/10, Loss: 0.5036, Acc: 0.7550, Val Loss: 0.4870, Val Acc: 0.7662


# Question 5: What is the mean of test loss for all the epochs for the model trained with augmentations?

In [13]:
mean_aug_val_loss = np.mean(history['val_loss'][10:20])
mean_aug_val_loss

np.float64(0.5909868313601954)

# Question 6: What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [14]:
mean_last5_aug_acc = np.mean(history['val_acc'][15:20])
mean_last5_aug_acc

np.float64(0.6985074626865672)